In [ ]:
import wandb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

api = wandb.Api(timeout=60)

In [ ]:
# shape 4,8,8,2, extension from 0 to 20 levels
sweep_ids=['wpcowdl5', 'j9lvvxjg', 'a5kr3muw', 'iat6x9bh']
untapped_potential = 'untapped-potential'
_active_weights_abs = "active-weights-abs"
_pparams = "pparams"
_untapped_potential = 'untapped-potential'

# with shape 4,4,4,4,2
if True:
    #sweep_ids=['t1mp5nhi', 'c3gin27l', '82wq0l59', 'vk3ftk2q','vpqib95x'] # 3 hidden layers
    sweep_ids=['f9zju9yb', '01ayiisq']  # one hidden layer
    _pparams = "meta/" + _pparams
    _untapped_potential = 'graph/' + _untapped_potential
    _active_weights_abs = "graph/" + _active_weights_abs

runs = []
for sweep_id in sweep_ids:
    runs += [run for run in api.sweep('concat_moons/' + sweep_id).runs]

#histories = [run.history() for run in runs]
configs = [run.config for run in runs]

In [ ]:
# single layer. network has 4, 20, 2 base shape
t1 = [26550,18054,12276,8346,5676,3858,2622,1782,1212,828,564,384,258,174,120]

# two layer. network has 4, 8, 8, 2 base shape
t2 = [170560,115591,78952,53815,36472,24640,16891,11440,7735,5320,3591,2491,1672,1147,775,520,352,247,160,112]

# three layer. 4, 4, 4, 4, 2 base shape
t3 = [26676,18236,12320,8316,5720,3956,2660,1856,1196,836,540,360,260,176,108,80,56]

pd.DataFrame.from_dict({'t1': reversed(t1), 't2': reversed(t2), 't3':reversed(t3)},orient='index').T

In [ ]:
shapes, level, pparams = [], [], []
for c in configs:
    shapes.append(c['model_shape'][1])
    level.append(c['extension_levels'])
    pparams.append(c['param_trajectory'][0])
pd.DataFrame({
    'shape' : shapes,
    'level': level,
    'pparams':pparams
}).sort_values('level').drop_duplicates().reset_index(drop=True).drop('level', axis=1).to_csv('1layer.csv')

In [ ]:
l1 = pd.read_csv('1layer.csv').rename(columns={'shape':'layer-1-shape', 'pparams': 'layer-1-prunable-parameters'})
l2 = pd.read_csv('2layer.csv').rename(columns={'shape':'layer-2-shape', 'pparams': 'layer-2-prunable-parameters'})
l3 = pd.read_csv('3layer.csv').rename(columns={'shape':'layer-3-shape', 'pparams': 'layer-3-prunable-parameters'})

In [ ]:
pd.merge(l1, 
    pd.merge(l2, l3, on='Unnamed: 0', how='outer'),
    on='Unnamed: 0',
    how='outer'
).drop('Unnamed: 0', axis=1).astype('Int64').to_csv('table.csv')


In [ ]:
df = pd.read_csv('table.csv')
df['L3-shape'] = df['L3-shape'].shift(-2)
df['L3-prunable-params'] = df['L3-prunable-params'].shift(-2)
df.at[15, 'L1-shape'] = 6507
df.at[16, 'L1-shape'] = 9569
df.at[15, 'L1-params'] = 39042
df.at[16, 'L1-params'] = 57414


df.at[15, 'L3-shape'] = 139
df.at[16, 'L3-shape'] = 169
df.at[15, 'L3-prunable-params'] = 39476
df.at[16, 'L3-prunable-params'] = 58136
df = df.astype('Int64')
df.columns = ['Lvl', 'shape(1)', 'param(1)', 'shape(2)', 'param(2)', 'shape(3)', 'param(3)']
df.to_latex(
    's.txt', 
    index=False,
    caption='In this table, the parameter trajectories and the corresponding hidden dimension of the network are displayed for each extension level. Parameter trajectory is in each respective "param" column and the number of hidden neurons per hidden layer in the column "hidden dim". At the extension level zero, the base model values are displayed.',
    label='tab:trajectory',
)